#Загрузка данных

In [ ]:
!git clone https://github.com/iurii-chiryshev/PalmEdu
!pip install -U --no-cache-dir gdown --pre
!gdown "https://drive.google.com/uc?id=193YTyFo8QuQsAMvi8oxNhNhicFSNmuW8"
!unzip "data.zip"

#Решение пошло

In [ ]:
from __future__ import print_function, division
from __future__ import absolute_import

import torch
from torch import nn
from torch.nn import init
from torch.utils.data import DataLoader
from torch.utils import data
from torchvision import transforms

import numpy as np
import pandas as pd

import os
import sys
import cv2
from collections import OrderedDict
import argparse
from datetime import datetime
import random
import math
import cv2
import json
import glob
import pathlib
from PIL import Image
from google.colab.patches import cv2_imshow

In [ ]:
"""Description for the script:
SSR-Net in Pytorch.
@see https://raw.githubusercontent.com/oukohou/SSR_Net_Pytorch/master/SSR_models/SSR_Net_model.py
"""

import torch
from torch import nn
from torch.nn import init


class SSRNet(nn.Module):
    def __init__(self, stage_num=[3, 3, 3], image_size=64,
                 class_range=101, lambda_index=1., lambda_delta=1.):
        super(SSRNet, self).__init__()
        self.image_size = image_size
        self.stage_num = stage_num
        self.lambda_index = lambda_index
        self.lambda_delta = lambda_delta
        self.class_range = class_range

        self.stream1_stage3 = nn.Sequential(
            nn.Conv2d(3, 32, 3, 1, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.AvgPool2d(2, 2)
        )
        self.stream1_stage2 = nn.Sequential(
            nn.Conv2d(32, 32, 3, 1, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.AvgPool2d(2, 2)
        )
        self.stream1_stage1 = nn.Sequential(
            nn.Conv2d(32, 32, 3, 1, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.AvgPool2d(2, 2),
            nn.Conv2d(32, 32, 3, 1, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            # nn.AvgPool2d(2, 2) # paper has this layer, but official codes don't.
        )

        self.stream2_stage3 = nn.Sequential(
            nn.Conv2d(3, 16, 3, 1, 1),
            nn.BatchNorm2d(16),
            nn.Tanh(),
            nn.MaxPool2d(2, 2)
        )
        self.stream2_stage2 = nn.Sequential(
            nn.Conv2d(16, 16, 3, 1, 1),
            nn.BatchNorm2d(16),
            nn.Tanh(),
            nn.MaxPool2d(2, 2)
        )
        self.stream2_stage1 = nn.Sequential(
            nn.Conv2d(16, 16, 3, 1, 1),
            nn.BatchNorm2d(16),
            nn.Tanh(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 16, 3, 1, 1),
            nn.BatchNorm2d(16),
            nn.Tanh(),
            # nn.MaxPool2d(2, 2) # paper has this layer, but official codes don't.
        )

        # fusion block
        self.funsion_block_stream1_stage_3_before_PB = nn.Sequential(
            nn.Conv2d(32, 10, 1, padding=0),
            nn.ReLU(),
            nn.AvgPool2d(8, 8)
        )
        self.funsion_block_stream1_stage_3_prediction_block = nn.Sequential(
            nn.Dropout(0.2, ),
            nn.Linear(10 * 4 * 4, self.stage_num[2]),
            nn.ReLU()
        )

        self.funsion_block_stream1_stage_2_before_PB = nn.Sequential(
            nn.Conv2d(32, 10, 1, padding=0),
            nn.ReLU(),
            nn.AvgPool2d(4, 4)
        )
        self.funsion_block_stream1_stage_2_prediction_block = nn.Sequential(
            nn.Dropout(0.2, ),
            nn.Linear(10 * 4 * 4, self.stage_num[1]),
            nn.ReLU()
        )

        self.funsion_block_stream1_stage_1_before_PB = nn.Sequential(
            nn.Conv2d(32, 10, 1, padding=0),
            nn.ReLU(),
            # nn.AvgPool2d(2, 2) # paper has this layer, but official codes don't.
        )
        self.funsion_block_stream1_stage_1_prediction_block = nn.Sequential(
            nn.Dropout(0.2, ),
            nn.Linear(10 * 8 * 8, self.stage_num[0]),
            nn.ReLU()
        )

        # stream2
        self.funsion_block_stream2_stage_3_before_PB = nn.Sequential(
            nn.Conv2d(16, 10, 1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(8, 8)
        )
        self.funsion_block_stream2_stage_3_prediction_block = nn.Sequential(
            nn.Dropout(0.2, ),
            nn.Linear(10 * 4 * 4, self.stage_num[2]),
            nn.ReLU()
        )

        self.funsion_block_stream2_stage_2_before_PB = nn.Sequential(
            nn.Conv2d(16, 10, 1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(4, 4)
        )
        self.funsion_block_stream2_stage_2_prediction_block = nn.Sequential(
            nn.Dropout(0.2, ),
            nn.Linear(10 * 4 * 4, self.stage_num[1]),
            nn.ReLU()
        )

        self.funsion_block_stream2_stage_1_before_PB = nn.Sequential(
            nn.Conv2d(16, 10, 1, padding=0),
            nn.ReLU(),
            # nn.MaxPool2d(2, 2) # paper has this layer, but official codes don't.
        )
        self.funsion_block_stream2_stage_1_prediction_block = nn.Sequential(
            nn.Dropout(0.2, ),
            nn.Linear(10 * 8 * 8, self.stage_num[0]),
            nn.ReLU()
        )

        self.stage3_FC_after_PB = nn.Sequential(
            nn.Linear(self.stage_num[0], 2 * self.stage_num[0]),
            nn.ReLU()
        )
        self.stage3_prob = nn.Sequential(
            nn.Linear(2 * self.stage_num[0], self.stage_num[0]),
            nn.ReLU()
        )
        self.stage3_index_offsets = nn.Sequential(
            nn.Linear(2 * self.stage_num[0], self.stage_num[0]),
            nn.Tanh()
        )
        self.stage3_delta_k = nn.Sequential(
            nn.Linear(10 * 4 * 4, 1),
            nn.Tanh()
        )

        self.stage2_FC_after_PB = nn.Sequential(
            nn.Linear(self.stage_num[0], 2 * self.stage_num[0]),
            nn.ReLU()
        )
        self.stage2_prob = nn.Sequential(
            nn.Linear(2 * self.stage_num[0], self.stage_num[0]),
            nn.ReLU()
        )
        self.stage2_index_offsets = nn.Sequential(
            nn.Linear(2 * self.stage_num[0], self.stage_num[0]),
            nn.Tanh()
        )
        self.stage2_delta_k = nn.Sequential(
            nn.Linear(10 * 4 * 4, 1),
            nn.Tanh()
        )

        self.stage1_FC_after_PB = nn.Sequential(
            nn.Linear(self.stage_num[0], 2 * self.stage_num[0]),
            nn.ReLU()
        )
        self.stage1_prob = nn.Sequential(
            nn.Linear(2 * self.stage_num[0], self.stage_num[0]),
            nn.ReLU()
        )
        self.stage1_index_offsets = nn.Sequential(
            nn.Linear(2 * self.stage_num[0], self.stage_num[0]),
            nn.Tanh()
        )
        self.stage1_delta_k = nn.Sequential(
            nn.Linear(10 * 8 * 8, 1),
            nn.Tanh()
        )
        self.init_params()

    def init_params(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.001)
                if m.bias is not None:
                    init.constant_(m.bias, 0)

    def forward(self, image_):
        feature_stream1_stage3 = self.stream1_stage3(image_)

        feature_stream1_stage2 = self.stream1_stage2(feature_stream1_stage3)

        feature_stream1_stage1 = self.stream1_stage1(feature_stream1_stage2)

        feature_stream2_stage3 = self.stream2_stage3(image_)

        feature_stream2_stage2 = self.stream2_stage2(feature_stream2_stage3)

        feature_stream2_stage1 = self.stream2_stage1(feature_stream2_stage2)

        feature_stream1_stage3_before_PB = self.funsion_block_stream1_stage_3_before_PB(feature_stream1_stage3)
        feature_stream1_stage2_before_PB = self.funsion_block_stream1_stage_2_before_PB(feature_stream1_stage2)
        feature_stream1_stage1_before_PB = self.funsion_block_stream1_stage_1_before_PB(feature_stream1_stage1)

        feature_stream2_stage3_before_PB = self.funsion_block_stream2_stage_3_before_PB(feature_stream2_stage3)
        feature_stream2_stage2_before_PB = self.funsion_block_stream2_stage_2_before_PB(feature_stream2_stage2)
        feature_stream2_stage1_before_PB = self.funsion_block_stream2_stage_1_before_PB(feature_stream2_stage1)

        embedding_stream1_stage3_before_PB = feature_stream1_stage3_before_PB.view(
            feature_stream1_stage3_before_PB.size(0), -1)
        embedding_stream1_stage2_before_PB = feature_stream1_stage2_before_PB.view(
            feature_stream1_stage2_before_PB.size(0), -1)
        embedding_stream1_stage1_before_PB = feature_stream1_stage1_before_PB.view(
            feature_stream1_stage1_before_PB.size(0), -1)

        embedding_stream2_stage3_before_PB = feature_stream2_stage3_before_PB.view(
            feature_stream2_stage3_before_PB.size(0), -1)
        embedding_stream2_stage2_before_PB = feature_stream2_stage2_before_PB.view(
            feature_stream2_stage2_before_PB.size(0), -1)
        embedding_stream2_stage1_before_PB = feature_stream2_stage1_before_PB.view(
            feature_stream2_stage1_before_PB.size(0), -1)
        stage1_delta_k = self.stage1_delta_k(
            torch.mul(embedding_stream1_stage1_before_PB, embedding_stream2_stage1_before_PB))
        stage2_delta_k = self.stage2_delta_k(
            torch.mul(embedding_stream1_stage2_before_PB, embedding_stream2_stage2_before_PB))
        stage3_delta_k = self.stage3_delta_k(
            torch.mul(embedding_stream1_stage3_before_PB, embedding_stream2_stage3_before_PB))

        embedding_stage1_after_PB = torch.mul(
            self.funsion_block_stream1_stage_1_prediction_block(embedding_stream1_stage1_before_PB),
            self.funsion_block_stream2_stage_1_prediction_block(embedding_stream2_stage1_before_PB))
        embedding_stage2_after_PB = torch.mul(
            self.funsion_block_stream1_stage_2_prediction_block(embedding_stream1_stage2_before_PB),
            self.funsion_block_stream2_stage_2_prediction_block(embedding_stream2_stage2_before_PB))
        embedding_stage3_after_PB = torch.mul(
            self.funsion_block_stream1_stage_3_prediction_block(embedding_stream1_stage3_before_PB),
            self.funsion_block_stream2_stage_3_prediction_block(embedding_stream2_stage3_before_PB))

        embedding_stage1_after_PB = self.stage1_FC_after_PB(embedding_stage1_after_PB)
        embedding_stage2_after_PB = self.stage2_FC_after_PB(embedding_stage2_after_PB)
        embedding_stage3_after_PB = self.stage3_FC_after_PB(embedding_stage3_after_PB)

        prob_stage_1 = self.stage1_prob(embedding_stage1_after_PB)
        index_offset_stage1 = self.stage1_index_offsets(embedding_stage1_after_PB)

        prob_stage_2 = self.stage2_prob(embedding_stage2_after_PB)
        index_offset_stage2 = self.stage2_index_offsets(embedding_stage2_after_PB)

        prob_stage_3 = self.stage3_prob(embedding_stage3_after_PB)
        index_offset_stage3 = self.stage3_index_offsets(embedding_stage3_after_PB)
        stage1_regress = prob_stage_1[:, 0] * 0
        stage2_regress = prob_stage_2[:, 0] * 0
        stage3_regress = prob_stage_3[:, 0] * 0
        for index in range(self.stage_num[0]):
            stage1_regress = stage1_regress + (
                        index + self.lambda_index * index_offset_stage1[:, index]) * prob_stage_1[:, index]
        stage1_regress = torch.unsqueeze(stage1_regress, 1)
        stage1_regress = stage1_regress / (self.stage_num[0] * (1 + self.lambda_delta * stage1_delta_k))

        for index in range(self.stage_num[1]):
            stage2_regress = stage2_regress + (
                        index + self.lambda_index * index_offset_stage2[:, index]) * prob_stage_2[:, index]
        stage2_regress = torch.unsqueeze(stage2_regress, 1)
        stage2_regress = stage2_regress / (self.stage_num[0] * (1 + self.lambda_delta * stage1_delta_k) *
                                           (self.stage_num[1] * (1 + self.lambda_delta * stage2_delta_k)))

        for index in range(self.stage_num[2]):
            stage3_regress = stage3_regress + (
                        index + self.lambda_index * index_offset_stage3[:, index]) * prob_stage_3[:, index]
        stage3_regress = torch.unsqueeze(stage3_regress, 1)
        stage3_regress = stage3_regress / (self.stage_num[0] * (1 + self.lambda_delta * stage1_delta_k) *
                                           (self.stage_num[1] * (1 + self.lambda_delta * stage2_delta_k)) *
                                           (self.stage_num[2] * (1 + self.lambda_delta * stage3_delta_k))
                                           )
        regress_class = (stage1_regress + stage2_regress + stage3_regress) * self.class_range
        regress_class = torch.squeeze(regress_class, 1)
        return regress_class


In [ ]:
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        x = x.transpose(3, 2).contiguous()
        return x.view(x.size(0), -1)


class ONet(nn.Module):
    def __init__(self):
        super(ONet, self).__init__()

        self.features = nn.Sequential(
            OrderedDict([
                ('conv1', nn.Conv2d(3, 32, 3, 1)),
                ('prelu1', nn.PReLU(32)),
                ('pool1', nn.MaxPool2d(3, 2, ceil_mode=True)),
                ('conv2', nn.Conv2d(32, 64, 3, 1)),
                ('prelu2', nn.PReLU(64)),
                ('pool2', nn.MaxPool2d(3, 2, ceil_mode=True)),
                ('conv3', nn.Conv2d(64, 64, 3, 1)),
                ('prelu3', nn.PReLU(64)),
                ('pool3', nn.MaxPool2d(2, 2, ceil_mode=True)),
                ('conv4', nn.Conv2d(64, 128, 2, 1)),
                ('prelu4', nn.PReLU(128)),
            ]))

        self.flatten = Flatten()
        self.conv5 = nn.Linear(3200, 256)  # 21632 if [128x128], 3200 if [64x64]
        self.drop5 = nn.Dropout(0.25)
        self.prelu5 = nn.PReLU(256)
        self.result = nn.Linear(256, 1) #

    def forward(self, x):
        x = self.features(x)
        x = self.flatten(x)
        x = self.conv5(x)
        x = self.drop5(x)

        x = self.result(x).view(-1)
        return x

In [ ]:
df = pd.read_csv(os.path.join(path,"label.csv"),sep=";")

In [ ]:
class AngleDatasets(data.Dataset):
    def __init__(self, path, transforms=None):
        self.path = path
        self.transforms = transforms
        df = pd.read_csv(os.path.join(path,"label.csv"),sep=";")
        self.list = df.values.tolist()

    def __getitem__(self, index):
        row = self.list[index]
        file_name = row[0]
        label = row[1]
        img = cv2.imread(os.path.join(self.path, file_name),cv2.IMREAD_COLOR)

        if self.transforms:
            [img, label] = self.transforms(img,label)
        return img, label

    def __len__(self):
        return len(self.list)


class Logger:
    def __init__(self, log_dir=None):
        if TENSORBOARD:
            self.writer = tf.summary.create_file_writer(log_dir)


    def scalar_summary(self, tag, value, step):
        if TENSORBOARD:
            with self.writer.as_default():
                tf.summary.scalar(tag, value, step=step)
                self.writer.flush()

In [ ]:
try:
    import tensorflow as tf
    TENSORBOARD = True
except ImportError:
    print('tensorflow not found')
    TENSORBOARD = False

In [ ]:
class Rotate(object):
    def __init__(self, angle_range = (-45,45), prob = 0.5, padding_color=0):
        """
        :param angle_range: Rotation angle in degrees. Positive values mean counter-clockwise rotation.
        :param prob:
        :param padding_color:
        """
        self.arange = angle_range
        self.prob = prob
        self.padding_color = padding_color


    def __call__(self, img, label):
        if random.random() < self.prob:
            # вращаем на случайный угол относительно центра изображения
            (h, w, c) = img.shape
            angle = random.random() * (self.arange[1] - self.arange[0]) + self.arange[0]
            rot_mat = cv2.getRotationMatrix2D((w//2,h//2), angle, 1.0)
            img = cv2.warpAffine(img, rot_mat, (w,h), flags=cv2.INTER_LINEAR,
                                 borderMode=cv2.BORDER_CONSTANT,
                                 borderValue = self.padding_color)
            label = angle + label
        return [img, label]

class VeritcalFlip(object):
    def __init__(self, prob = 0.5):
        self.prob = prob
    def __call__(self, image, label):
        if random.random() < self.prob:
            image = cv2.flip(image, 0) # veritcal flip
            label = 360 - label
        return [image, label]

class Normalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, image, label):
        image = image.astype(np.float32)
        for i in range(3):
            image[:,:,i] -= self.mean[i]
        for i in range(3):
            image[:,:, i] /= self.std[i]

        return [image, label]

class Resize(object):
    def __init__(self, size=(64,64)):
        self.size = size

    def __call__(self, image, label):
        image = cv2.resize(image, self.size)
        return [image, label]

class ToTensor(object):
    def __call__(self, image, label):

        image = image.transpose((2,0,1)) # change [H,W,C] to [C,H,W]

        return [image, label]

class Compose(object):
    """Composes several transforms together.
    """

    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, *args):
        for t in self.transforms:
            args = t(*args)
        return args

In [ ]:
def save_checkpoint(state, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    print('Save checkpoint to {0:}'.format(filename))


def adjust_learning_rate(optimizer, initial_lr, step_index):
    lr = initial_lr * (0.1 ** (step_index))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr


def train(net, trainloader, criterion, optimizer, device):
    net.train()
    train_loss = []
    train_mae = []

    for imgs, labels in trainloader:
        optimizer.zero_grad()
        imgs, labels = imgs.to(device), labels.to(torch.float32).to(device)
        predicts = net(imgs)
        loss = criterion(predicts,labels)
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())
        mae = torch.mean(torch.abs(predicts.view(-1) - labels.view(-1)))
        train_mae.append(mae.item())

    return np.mean(train_loss), np.mean(train_mae)


def validate(net, valloader, criterion,device):
    net.eval()
    val_mae = []
    val_loss = []
    for img, labels in valloader:
        img = img.to(device)
        labels = labels.to(torch.float32).to(device)
        with torch.no_grad():
            predicts = net(img)
            loss = criterion(predicts, labels)
            val_loss.append(loss.item())

        mae = torch.mean(torch.abs(labels.view(-1) - predicts.view(-1)))
        val_mae.append(mae.item())
    return np.mean(val_loss), np.mean(val_mae)

In [ ]:
class Data_args():
  def __init__(self, workers = 10, base_lr=0.002, weight_decay=5e-4, step="50,75,88,120", start_epoch=1,
               end_epoch = 350, snapshot='./models/checkpoint/snapshot/', tensorboard="./models/checkpoint/tensorboard", resume='',train_path='/content/train', 
               test_path='/content/test',train_batchsize=32, test_batchsize=32,  input_size=64):
    
    self.workers = workers
    self.base_lr = base_lr
    self.weight_decay = weight_decay
    self.step = step
    self.start_epoch = start_epoch
    self.end_epoch = end_epoch
    self.snapshot = snapshot
    self.tensorboard = tensorboard
    self.resume = resume
    self.train_path = train_path
    self.test_path = test_path
    self.train_batchsize = train_batchsize
    self.test_batchsize = test_batchsize
    self.input_size = input_size

In [ ]:
args = Data_args()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

net = SSRNet(image_size=64,class_range=360)
#net = ONet()
net.to(device)

dt = datetime.now().strftime("%m%d%Y_%H%M")
snapshot_path = os.path.join(args.snapshot, dt)
tensorboard_path = os.path.join(args.tensorboard, dt)

if not os.path.exists(snapshot_path):
    os.makedirs(snapshot_path)

if not os.path.exists(tensorboard_path):
    os.makedirs(tensorboard_path)
logger = Logger(tensorboard_path)
# TODO tensorboard --logdir models/logs/09092022_1212


# super params
step_epoch = [int(x) for x in args.step.split(',')]

criterion = torch.nn.L1Loss()
loss_name = criterion.__class__.__name__
criterion = criterion.to(device)
cur_lr = args.base_lr
optimizer = torch.optim.Adam(
    net.parameters(),
    lr=args.base_lr,
    weight_decay=args.weight_decay)

# dataset
train_transform = Compose([
    #VeritcalFlip(),
    #Rotate(angle_range = (-90,90), prob = 1.0),
    Resize((args.input_size,args.input_size)),
    Normalize(mean=(127.5,127.5,127.5),std=(127.5,127.5,127.5)),
    #Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ToTensor()
])
test_transform = Compose([
    #VeritcalFlip(),
    #Rotate(angle_range=(-90, 90), prob=1.0),
    Resize((args.input_size, args.input_size)),
    Normalize(mean=(127.5, 127.5, 127.5), std=(127.5, 127.5, 127.5)),
    #Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ToTensor()
])

train_datasets = AngleDatasets(args.train_path,transforms=train_transform)
trainloader = DataLoader(
    train_datasets,
    batch_size=args.train_batchsize,
    shuffle=True,
    num_workers=args.workers,
    drop_last=True)

test_datasets = AngleDatasets(args.test_path,transforms=test_transform)
valloader = DataLoader(
    test_datasets,
    batch_size=args.test_batchsize,
    shuffle=False,
    num_workers=args.workers,
    drop_last=True)

step_index = 0

for epoch in range(args.start_epoch, args.end_epoch + 1):
  train_loss, train_MAE = train(net, trainloader,
                          criterion, optimizer, device)
  val_loss, val_MAE = validate(net,valloader,criterion,device)
  if epoch in step_epoch:
      step_index += 1
      cur_lr = adjust_learning_rate(optimizer, args.base_lr, step_index)

  print('Epoch: %d,  train_loss:%6.4f, val_loss:%8.6f, train_MAE:%6.4f, val_MAE:%8.6f, lr:%8.6f' % (epoch, train_loss, val_loss, train_MAE, val_MAE, cur_lr))
  filename = os.path.join(
      str(snapshot_path), "checkpoint_epoch_" + str(epoch) + '.pth')
  save_checkpoint(net.state_dict(), filename)

  info = {
      "{}/Train".format(loss_name): train_loss,
      "{}/Val".format(loss_name): val_loss,

      'MSE/Train': train_MAE,
      'MSE/Val': val_MAE,

      "Learning rate": cur_lr
  }

  for tag, value in info.items():
      logger.scalar_summary(tag, value, epoch)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 1,  train_loss:35.0345, val_loss:8.754964, train_MAE:35.0345, val_MAE:8.754964, lr:0.002000
Save checkpoint to ./models/checkpoint/snapshot/01172023_1150/checkpoint_epoch_1.pth
Epoch: 2,  train_loss:13.4185, val_loss:16.338616, train_MAE:13.4185, val_MAE:16.338616, lr:0.002000
Save checkpoint to ./models/checkpoint/snapshot/01172023_1150/checkpoint_epoch_2.pth
Epoch: 3,  train_loss:12.7655, val_loss:6.496736, train_MAE:12.7655, val_MAE:6.496736, lr:0.002000
Save checkpoint to ./models/checkpoint/snapshot/01172023_1150/checkpoint_epoch_3.pth
Epoch: 4,  train_loss:10.4367, val_loss:4.593840, train_MAE:10.4367, val_MAE:4.593840, lr:0.002000
Save checkpoint to ./models/checkpoint/snapshot/01172023_1150/checkpoint_epoch_4.pth
Epoch: 5,  train_loss:10.4038, val_loss:6.323503, train_MAE:10.4038, val_MAE:6.323503, lr:0.002000
Save checkpoint to ./models/checkpoint/snapshot/01172023_1150/checkpoint_epoch_5.pth
Epoch: 6,  train_loss:9.9207, val_loss:7.234535, train_MAE:9.9207, val_MAE:7.2

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-53eee403dbd9>", line 73, in <module>
    train_loss, train_MAE = train(net, trainloader,
  File "<ipython-input-10-0f6214c3531c>", line 18, in train
    for imgs, labels in trainloader:
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 628, in __next__
    data = self._next_data()
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1316, in _next_data
    idx, data = self._get_data()
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1282, in _get_data
    success, data = self._try_get_data()
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1120, in _try_get_data
    data = self._data_queue.get(timeout=timeout)
  File "/usr/lib/python

KeyboardInterrupt: ignored

In [ ]:
print(validate(net,valloader,criterion,device))

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


(1.8441887230708682, 1.8441887230708682)
